In [34]:
import argparse
import json
import math
import os
import numpy as np
from typing import List, Tuple, Dict

from tqdm import tqdm



EARTH_RADIUS_METERS = 6.378137e6
REFERENCE_COORDINATES = {
    "zero": [0.000000000000, 0.000000000000]
}

dataroot = '/disk/ml/datasets/ONCE/data_root/data/'

oxts_folder = '/disk/vanishing_data/ju878/once_kitti_test/oxts/'
poses_folder = '/disk/vanishing_data/ju878/once_kitti_test/poses/'

if not os.path.exists(oxts_folder):
        os.makedirs(oxts_folder)
if not os.path.exists(poses_folder):
        os.makedirs(poses_folder)

In [3]:
def get_poses(frames) -> List[dict]:
    """
    Return all ego poses for the current scene.
    :param nusc: The NuScenes instance to load the ego poses from.
    :param scene_token: The token of the scene.
    :return: A list of the ego pose dicts.
    """
    pose_list = []
    for index, frame in enumerate(frames):
        pose_list.append(frame['pose'])
    
    return pose_list

In [9]:
def get_coordinate(ref_lat: float, ref_lon: float, bearing: float, dist: float) -> Tuple[float, float]:
    """
    Using a reference coordinate, extract the coordinates of another point in space given its distance and bearing
    to the reference coordinate. For reference, please see: https://www.movable-type.co.uk/scripts/latlong.html.
    :param ref_lat: Latitude of the reference coordinate in degrees, ie: 42.3368.
    :param ref_lon: Longitude of the reference coordinate in degrees, ie: 71.0578.
    :param bearing: The clockwise angle in radians between target point, reference point and the axis pointing north.
    :param dist: The distance in meters from the reference point to the target point.
    :return: A tuple of lat and lon.
    """
    lat, lon = math.radians(ref_lat), math.radians(ref_lon)
    angular_distance = dist / EARTH_RADIUS_METERS
    
    target_lat = math.asin(
        math.sin(lat) * math.cos(angular_distance) + 
        math.cos(lat) * math.sin(angular_distance) * math.cos(bearing)
    )
    target_lon = lon + math.atan2(
        math.sin(bearing) * math.sin(angular_distance) * math.cos(lat),
        math.cos(angular_distance) - math.sin(lat) * math.sin(target_lat)
    )
    return math.degrees(target_lat), math.degrees(target_lon)

In [43]:
def derive_latlon(location: str, poses: List[Dict[str, float]], scene_name, frames) -> List[Dict[str, float]]:
    """
    For each pose value, extract its respective lat/lon coordinate and timestamp.
    
    This makes the following two assumptions in order to work:
        1. The reference coordinate for each map is in the south-western corner.
        2. The origin of the global poses is also in the south-western corner (and identical to 1).

    :param location: The name of the map the poses correspond to, ie: 'boston-seaport'.
    :param poses: All nuScenes egopose dictionaries of a scene.
    :return: A list of dicts (lat/lon coordinates and timestamps) for each pose.
    """
    assert location in REFERENCE_COORDINATES.keys(), \
        f'Error: The given location: {location}, has no available reference.'
        
    oxts_scene_folder = os.path.join(oxts_folder, scene_name)
    oxts = []
    
    # activate if oxts should be safed
    #if not os.path.exists(oxts_scene_folder):
    #    os.makedirs(oxts_scene_folder)
    
    reference_lat, reference_lon = REFERENCE_COORDINATES[location]
    for i, p in enumerate(poses):
        x, y, z = p[4:]
        if y == 0:
            bearing = 0.0
        else:
            bearing = math.atan(x / y)
        distance = math.sqrt(x**2 + y**2)
        lat, lon = get_coordinate(reference_lat, reference_lon, bearing, distance)
        
        r1, r2, r3, w = p[:4]
        roll = math.atan2(2 * (w * r1 + r2 * r3), 1 - 2 * (math.pow(r1, 2) + math.pow(r2, 2)))
        pitch = math.asin(2 * (w * r2 - r3 * r1))
        yaw = math.atan2(2 * (w * r3 + r1 * r2), 1 - 2 * (math.pow(r2, 2) + math.pow(r3, 2)))
        
        oxts.append([lat, lon, z, roll, pitch, yaw])
        
        # activate if oxts should be safed
        #with open(os.path.join(oxts_scene_folder, frames[i]['frame_id'] + '.txt'), 'w') as f:
        #    f.write(f'{lat} {lon} {z} {roll} {pitch} {yaw}')
    return oxts

In [20]:
def load_oxts_data(scene_token: str, base_dir):
    
    scene_rec = nusc.get('scene', scene_token)
    sample_rec = nusc.get('sample', scene_rec['first_sample_token'])
    sd_rec = nusc.get('sample_data', sample_rec['data']['LIDAR_TOP'])
    
    number_of_samples = 1
    while sd_rec['next'] != '':
        number_of_samples += 1
        sd_rec = nusc.get('sample_data', sd_rec['next'])
        
    oxts_data = np.zeros((number_of_samples, OXTS_LINE_LEN))
    sd_rec = nusc.get('sample_data', sample_rec['data']['LIDAR_TOP'])

    for i in range(number_of_samples):
        if i != 0:
            sd_rec = nusc.get('sample_data', sd_rec['next'])
        ego_pose = nusc.get('ego_pose', sd_rec['token'])
        oxts_data[i] = np.loadtxt(os.path.join(base_dir, ego_pose['token'] + '.txt'))

    return oxts_data

In [21]:
def lat_lon_to_mercator(lat, lon, scale):
    """
    Converts lat/lon coordinates to mercator coordinates using Mercator scale.
    """
    ER = 6378137
    mx = scale * lon * math.pi * ER / 180
    my = scale * ER * math.log(math.tan((90 + lat) * math.pi / 360))
    return mx, my


def lat_to_scale(lat):
    """
    Compute Mercator scale from latitude
    """
    return math.cos(lat * math.pi / 180.0)

In [22]:
def gps_imu_to_pose(gps_imu, scale):
    """
    Compute pose from GPS/IMU data
    """
    t = np.zeros(3)
    t[0], t[1] = lat_lon_to_mercator(gps_imu[0], gps_imu[1], scale)
    t[2] = gps_imu[2]  # altitude
    rx = gps_imu[3]  # roll
    ry = gps_imu[4]  # pitch
    rz = gps_imu[5]  # heading
    Rx = np.array([[1, 0, 0], [0, math.cos(rx), -math.sin(rx)], [0, math.sin(rx), math.cos(rx)]])
    Ry = np.array([[math.cos(ry), 0, math.sin(ry)], [0, 1, 0], [-math.sin(ry), 0, math.cos(ry)]])
    Rz = np.array([[math.cos(rz), -math.sin(rz), 0], [math.sin(rz), math.cos(rz), 0], [0, 0, 1]])
    R = Rz.dot(Ry).dot(Rx)
    T = np.identity(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T

In [32]:
def oxts_to_pose(oxts_info):
    """
    Converts a list of oxts measurements into metric poses,
    starting at (0,0,0) meters, OXTS coordinates are defined as
    x = forward, y = right, z = down (see OXTS RT3000 user manual)
    afterwards, pose{i} contains the transformation which takes a
    3D point in the i'th frame and projects it into the oxts
    coordinates of the first frame.
    """
    # Compute scale from first lat value
    scale = lat_to_scale(oxts_info[0][0])
    Tr_0_inv = None
    poses = np.zeros((len(oxts_info), 12))
    for i, oxts in enumerate(oxts_info):
        T = gps_imu_to_pose(oxts, scale)
        # Normalize translation and rotation (start at 0/0/0)
        if Tr_0_inv is None:
            Tr_0_inv = np.linalg.inv(T)

        pose = Tr_0_inv.dot(T)
        poses[i] = pose[:3, :].reshape(12)
    return poses

In [44]:
for scene in range(0, 581):
    scene_name = '{:0>6}'.format(scene)
    once_json_file = os.path.join(dataroot, scene_name, scene_name + '.json')
    with open(once_json_file, 'r') as f:
        once_json = json.load(f)
     
    frames = once_json['frames']
    
    once_poses = get_poses(frames)  # For each pose, we will extract the corresponding coordinate.
    # Compute and store coordinates.
    oxts = derive_latlon('zero', once_poses, scene_name, frames)
    
    kitti_poses = oxts_to_pose(oxts)
    np.savetxt(os.path.join(poses_folder, scene_name + '.txt'), kitti_poses)
